In [1]:
from arcgis.gis import *
import itertools
gis = GIS("https://ps-dbs.maps.arcgis.com", "bhorne_ps")

In [2]:
#Identifying users and siging into AGOL (Both users need to have Administrator role)
orig_userid = "bhorne_PS"
new_userid = "bhorne_PS_DBS"
olduser = gis.users.get(orig_userid)

In [3]:
gis.users.me

<User username:bhorne_PS>

In [5]:
#Transferring the ownership of ALL GROUPS owned by this user
print("\n***Transferring the ownership of groups...")
usergroups = olduser.groups
for group in usergroups:
    grp = gis.groups.get(group['id'])
    if (grp.owner == orig_userid):
        grp.reassign_to(new_userid)
    else:
        grp.add_users(new_userid)

print("The ownership of groups was successfully transferred!")
# usergroups


***Transferring the ownership of groups...
The ownership of groups was successfully transferred!


In [6]:
#Transferring items in folders        
print("\n***Transferring the ownership of items in selected folders...")

folders = olduser.folders
num_list=[]
for folder in folders:
    gis.content.create_folder(folder['title'], new_userid)
    folderitems = olduser.items(folder=folder['title'])
    num_items=int(len(folderitems))
    num_list.append(num_items)
    print("Transferring "+ str(num_items) + " item(s) from folder: "+folder['title'])
    for item in folderitems:
        if 'View Service' in item.typeKeywords:
            print(item.title + "This View will automatically be transferred")
        elif 'Microsoft' in item.type:
            print(item.title + " - Forget it")
        elif 'Code Attachment' in item.type:
            print("Forget This type")
        else:
            itemsharing=item.shared_with
            item.unshare(groups=itemsharing['groups'])
            item.reassign_to(new_userid, target_folder=folder['title'])
            item.share(everyone=itemsharing['everyone'], org=itemsharing['org'], groups=itemsharing['groups'], allow_members_to_edit=True)
            print(f"{item} was reassigned")

    def sum_list(num_list):
        sum = 0
        for x in num_list:
            sum += x
        return sum
    total_items=sum_list(num_list)

print("\nThe ownership of " +str(total_items) + " items was successfully transferred!")


***Transferring the ownership of items in selected folders...
Folder already exists.
Transferring 5 item(s) from folder: Survey-Dragon Sightings
<Item title:"Dragon Sightings" type:Feature Layer Collection owner:bhorne_PS_DBS> was reassigned
<Item title:"Dragon Sightings" type:Form owner:bhorne_PS_DBS> was reassigned
<Item title:"Dragon Sightings" type:Web Map owner:bhorne_PS_DBS> was reassigned
<Item title:"Dragon Sightings" type:Feature Layer Collection owner:bhorne_PS_DBS> was reassigned
<Item title:"DrogonWinterfell8x01" type:Image owner:bhorne_PS_DBS> was reassigned
Transferring 4 item(s) from folder: Survey-Fire Analysis
<Item title:"Fire Analysis" type:Form owner:bhorne_PS_DBS> was reassigned
<Item title:"Fire Analysis" type:Feature Layer Collection owner:bhorne_PS_DBS> was reassigned
<Item title:"Fire Analysis_stakeholder" type:Feature Layer Collection owner:bhorne_PS_DBS> was reassigned
<Item title:"Fire Analysis" type:Feature Layer Collection owner:bhorne_PS_DBS> was reassig

In [15]:
#Transferring items in the root folder   
print("\n***Transferring the ownership of items in the root folder...")
userrootcontent = olduser.items()
num_root_items=int(len(userrootcontent))
rootfoldername=olduser.username+"_RootFolderNew"
gis.content.create_folder(rootfoldername, new_userid)
root_groups_list=[]
# print(userrootcontent)

for item in userrootcontent:
    print(item)
    rootitemsharing=item.shared_with
    root_group=rootitemsharing['groups']
    root_groups_list.append(root_group)
    valueToBeRemoved = []
    try:
        while True:
            root_groups_list.remove(valueToBeRemoved)
    except ValueError:
        pass

    root_groups= list(itertools.chain(*root_groups_list))

    for group in root_groups:
        grop = gis.groups.get(group['id'])
        if (grop.owner == orig_userid):
            grop.reassign_to(new_userid)
        else:
            grop.add_users(new_userid)
            grop.remove_users(orig_userid)
    if 'View Service' in item.typeKeywords:
        print(item.title + "This View will automatically be transferred")
    elif 'Microsoft' in item.type:
        print(item.title + " - Forget it")
    elif 'Code Attachment' in item.type:
        print("Forget This type")
    else: 
        item.unshare(groups=rootitemsharing['groups'])
        item.reassign_to(new_userid, target_folder=rootfoldername)
        item.share(everyone=rootitemsharing['everyone'], org=rootitemsharing['org'], groups=rootitemsharing['groups'], allow_members_to_edit=True)

# print("The ownership of "+ str(num_root_items)+" items in Root Folder was transferred successfully!")

# print ("\nTotal number of items transferred: "+str(total_items+num_root_items)+" items") 

# print("\n***Process was completed!***")


***Transferring the ownership of items in the root folder...
Folder already exists.
<Item title:"USBP Sector AOR" type:Feature Layer Collection owner:bhorne_PS>
<Item title:"ArcGIS Online Administration" type:Notebook owner:bhorne_PS>
<Item title:"Item Management Dashboard - Sample" type:Dashboard owner:bhorne_PS>
<Item title:"Only Canada" type:Form owner:bhorne_PS>
<Item title:"Westeros___Fire_and_Blood_WTL1" type:Service Definition owner:bhorne_PS>
<Item title:"Westeros___Fire_and_Blood_WTL1" type:Map Image Layer owner:bhorne_PS>
<Item title:"GOT_Continents" type:Service Definition owner:bhorne_PS>
<Item title:"GOT_Continents" type:Feature Layer Collection owner:bhorne_PS>
<Item title:"Item Management Dashboard" type:Dashboard owner:bhorne_PS>
<Item title:"OES Wildfire Situational Awareness Dashboard - Public Desktop Version" type:Dashboard owner:bhorne_PS>
<Item title:"Test_Layer" type:Feature Layer Collection owner:bhorne_PS>
